In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import download_loader
from llama_index.core import VectorStoreIndex, ServiceContext, Document, StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

In [ ]:
import pandas as pd
from typing import Any, Dict
import logging
import json

In [ ]:
import llm_functions as lf

In [ ]:
import sys
sys.path.append('../libraries/')

In [ ]:
import neo4j_lib as nl
import claude_prompts as cp

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
MEMORY = ChatMemoryBuffer.from_defaults(token_limit=8192)
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=4096)
llm = Ollama(model="llama3.1:latest", temperature=0, max_tokens=8192, request_timeout=120.0)

In [ ]:
def create_chat_engine(advert):
    
    if advert:
        documents = [Document(text=advert)]

        index = VectorStoreIndex.from_documents(documents)
        return index.as_chat_engine(
            chat_mode="context",
            llm=llm,
            memory=MEMORY,
            system_prompt=(
                "A a career forensic analyst you have deep insight into crime and criminal activity especially human trafficking.  "
                "Your express goal is to investigate online recruitment advert and extract pertinent factual detail."
            ),
        )
    else:
        st.error(f"Failed to extract text from URL: {advert}")
        return None

In [ ]:

def analyse_advert(chat_engine: Any, advert: str, prompt_name: str) -> Dict[str, Any]:
    """
    Analyze an advertisement using a chat engine and a specific prompt.

    Args:
        chat_engine: The chat engine to use for analysis.
        advert: The advertisement text to analyze.
        prompt_name: The name of the prompt to use.

    Returns:
        A dictionary containing the analysis results, or a default dictionary if analysis fails.
    """
    if not chat_engine:
        logger.error("Chat engine is not initialized")
        return {
            "result": "error",
            "evidence": [],
            "explanation": "Chat engine not initialized",
            "confidence": 0.0,
        }

    try:
        prompt = cp.CLAUDE_PROMPTS.get(prompt_name) + cp.ANALYSIS_STR
        if not prompt:
            raise ValueError(f"Invalid prompt name: {prompt_name}")

        response = chat_engine.chat(prompt + cp.ANALYSIS_STR)
        logger.info(f"Response to {prompt_name}: {response.response}")

        # Extract and parse JSON from the response
        json_str = response.response.strip("`").strip()
        if json_str.startswith("json"):
            json_str = json_str[4:]  # Remove 'json' prefix if present
        parsed_response = json.loads(json_str)

        analysis_response = nl.AnalysisResponse(**parsed_response)
        # Standardize the output
        return analysis_response

    except Exception as e:
        logger.error(f"Error processing advert, prompt {prompt_name}: {str(e)}")
        return {
            "result": "error",
            "evidence": [],
            "explanation": str(e),
            "confidence": 0.0,
        }


In [ ]:

def analyse_advert_with_prompt(chat_engine: Any, advert: str, prompt: str) -> Dict[str, Any]:
    """
    Analyze an advertisement using a chat engine and a specific prompt.

    Args:
        chat_engine: The chat engine to use for analysis.
        advert: The advertisement text to analyze.
        prompt_name: The name of the prompt to use.

    Returns:
        A dictionary containing the analysis results, or a default dictionary if analysis fails.
    """
    if not chat_engine:
        logger.error("Chat engine is not initialized")
        return {
            "result": "error",
            "evidence": [],
            "explanation": "Chat engine not initialized",
            "confidence": 0.0,
        }

    try:
        complete_prompt = prompt + cp.ANALYSIS_STR
        if not prompt:
            raise ValueError(f"Invalid prompt name: {prompt_name}")

        response = chat_engine.chat(complete_prompt)
        logger.info(f"Response to {prompt_name}: {response.response}")

        # Extract and parse JSON from the response
        json_str = response.response.strip("`").strip()
        if json_str.startswith("json"):
            json_str = json_str[4:]  # Remove 'json' prefix if present
        parsed_response = json.loads(json_str)

        analysis_response = nl.AnalysisResponse(**parsed_response)
        # Standardize the output
        return analysis_response

    except Exception as e:
        logger.error(f"Error processing advert, prompt {prompt_name}: {str(e)}")
        return {
            "result": "error",
            "evidence": [],
            "explanation": str(e),
            "confidence": 0.0,
        }


In [ ]:
unprofessional_writing_prompt = (
        "Analyze the following job advertisement for signs of unprofessional writing such as poor grammar or spelling. "
        "Accept that missing spaces or words not separated by a space or spaces in the text, such as 'andcleaners', 'towork', is NOT a sign of unprofessional writing. "
        "Return your analysis in the following JSON format:"
    ),

In [ ]:
prompt_name = "assure_prompt"
query = """
MATCH (g:Group)-[:HAS_POSTING]-(posting:Posting)-[has_analysis:HAS_ANALYSIS {type: $prompt_name}]-(analysis:Analysis)
WHERE analysis.result = 'yes'
RETURN ID(g) AS groupIDn, g.name AS group_name, ID(posting) AS IDn, posting.post_url as url, posting.post_id AS post_id, posting.text AS advert, analysis.result as verdict, analysis.explanation as explanation
"""
parameters = {"prompt_name": prompt_name}
adverts = pd.DataFrame(nl.execute_neo4j_query(query, parameters))

In [ ]:
adverts.loc[0]['url'], adverts.loc[0]['IDn'], adverts.loc[0]['group_name']

In [ ]:
adverts.loc[0]['explanation']

In [ ]:
IDn = 573388
advert = nl.get_neo4j_advert(IDn)

In [ ]:
advert

In [ ]:
chat_engine = create_chat_engine(advert)

In [ ]:
advert_analysis = analyse_advert(chat_engine, advert, 'assure_prompt')

In [ ]:
prompt =  cp.CLAUDE_PROMPTS.get(prompt_name)

In [ ]:
advert_analysis =  analyse_advert_with_prompt(chat_engine, advert, prompt)

In [ ]:
advert_analysis

In [ ]:
unprofessional_writing_prompt[0]

In [ ]:
llm = Ollama(model="llama3.1", temperature=0, max_tokens=8192)

In [ ]:
llm = Ollama(model="llama3.1:latest", temperature=0, max_tokens=8192, request_timeout=120.0)